<a href="https://colab.research.google.com/github/llpk79/DS-Unit-1-Sprint-4-Statistical-Tests-and-Experiments/blob/master/Paul_K_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 3 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [185]:
names = col_names = ['party','Disabled Infants', 'Water Project', 'Budget Resolution', 'Physician Fee', 'El Salvador Aid', 'Religion-Schools', 'Anti-Satellite Test', 'Nicaragua Contras Aid', 'MX Missile', 'Immigration', 'Synfuels Cutback', 'Education Spending', 'Superfund Right To Sue', 'Crime', 'Duty Free Exports', 'Export Admin S. Africa']
gov = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data',
                  na_values='?', header=None, names=names)
gov.head()

,party,Disabled Infants,Water Project,Budget Resolution,Physician Fee,El Salvador Aid,Religion-Schools,Anti-Satellite Test,Nicaragua Contras Aid,MX Missile,Immigration,Synfuels Cutback,Education Spending,Superfund Right To Sue,Crime,Duty Free Exports,Export Admin S. Africa
0,republican,n,y,n,y,y,y,n,n,n,y,NaN,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,NaN
2,democrat,NaN,y,y,NaN,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,NaN,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,NaN,y,y,y,y


In [0]:
reps = gov[gov['party'] == 'republican']
dems = gov[gov['party'] == 'democrat']

In [0]:
reps.head()

,party,issue_1,issue_2,issue_3,issue_4,issue_5,issue_6,issue_7,issue_8,issue_9,issue_10,issue_11,issue_12,issue_13,issue_14,issue_15,issue_16
0,republican,n,y,n,y,y,y,n,n,n,y,NaN,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,NaN
7,republican,n,y,n,y,y,y,n,n,n,n,n,n,y,y,NaN,y
8,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,y
10,republican,n,y,n,y,y,n,n,n,n,n,NaN,NaN,y,y,n,n


In [0]:
dems.head()

,party,issue_1,issue_2,issue_3,issue_4,issue_5,issue_6,issue_7,issue_8,issue_9,issue_10,issue_11,issue_12,issue_13,issue_14,issue_15,issue_16
2,democrat,NaN,y,y,NaN,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,NaN,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,NaN,y,y,y,y
5,democrat,n,y,y,n,y,y,n,n,n,n,n,n,y,y,y,y
6,democrat,n,y,n,y,y,y,n,n,n,n,n,n,NaN,y,y,y


In [0]:
def convert_to_bool(x):
  if str(x) == 'y':
    return 1
  if str(x) == 'n':
    return 0
  else:
    return x

In [0]:
dems = dems.applymap(convert_to_bool)
reps = reps.applymap(convert_to_bool)

In [0]:
reps.head()

,party,Disabled Infants,Water Project,Budget Resolution,Physician Fee,El Salvador Aid,Religion-Schools,Anti-Satellite Test,Nicaragua Contras Aid,MX Missile,Immigration,Synfuels Cutback,Education Spending,Superfund Right To Sue,Crime,Duty Free Exports,Export Admin S. Africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
7,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,1.0
8,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
10,republican,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,1.0,0.0,0.0


In [0]:
dems.head()

,party,Disabled Infants,Water Project,Budget Resolution,Physician Fee,El Salvador Aid,Religion-Schools,Anti-Satellite Test,Nicaragua Contras Aid,MX Missile,Immigration,Synfuels Cutback,Education Spending,Superfund Right To Sue,Crime,Duty Free Exports,Export Admin S. Africa
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0
5,democrat,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
6,democrat,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,1.0,1.0


In [0]:
def fill_mode(column_names, frame):
  for name in column_names[1:]:
    frame[name].fillna(frame[name].mode()[0], inplace=True)


In [0]:
reps_zero = reps.copy()
dems_zero = dems.copy()

In [0]:
def fill_zero(column_names, frame):
  for name in column_names[1:]:
    frame[name].fillna(0, inplace=True)

In [0]:
fill_zero(names, reps_zero)
fill_zero(names, dems_zero)

In [0]:
dems_zero

In [0]:
reps_mode = reps.copy()
dems_mode = dems.copy()

In [0]:
fill_mode(names, reps_mode)
fill_mode(names, dems_mode)

In [0]:
from scipy.stats import ttest_ind

In [0]:
def do_ttests(column_names, dems, reps):
  # Make some empty lists.
  dems_support, reps_support, bipartisan = [], [], []
  
  # Iterate though issue columns.
  for name in column_names[1:]:
    
    # Get statistic and p value for each column.
    stat, pval = ttest_ind(dems[name], reps[name], nan_policy='omit')
    
    # Set a boolean indicating if p value is below threshold.
    sig = pval <= 0.01
    if sig:
      
      # If dems support and reps do not, add to dems list.
      if stat > 0:
        dems_support.append(name)
        
      # If republicans support and dems do not, add to reps list.
      elif stat < 0:
        reps_support.append(name)
        
    # If no significant difference found, add to bipartisan list.
    else:
      bipartisan.append(name)
  print('Dem supported issues: {}'.format(', '.join(dems_support)))
  print('Rep supported issues: {}'.format(', '.join(reps_support)))
  print('Bipartisan issues: {}'.format(', '.join(bipartisan)))


In [245]:
do_ttests(names, reps, dems)

Dem supported issues: Physician Fee, El Salvador Aid, Religion-Schools, Education Spending, Superfund Right To Sue, Crime
Rep supported issues: Disabled Infants, Budget Resolution, Anti-Satellite Test, Nicaragua Contras Aid, MX Missile, Synfuels Cutback, Duty Free Exports, Export Admin S. Africa
Bipartisan issues: Water Project, Immigration


In [202]:
do_ttests(names, reps_zero, dems_zero)

Dem supported issues: ['Physician Fee', 'El Salvador Aid', 'Religion-Schools', 'Education Spending', 'Superfund Right To Sue', 'Crime']
Rep supported issues: ['Disabled Infants', 'Budget Resolution', 'Anti-Satellite Test', 'Nicaragua Contras Aid', 'MX Missile', 'Duty Free Exports']
Bipartisan issues: ['Water Project', 'Immigration', 'Export Admin S. Africa']


In [203]:
do_ttests(names, reps_mode, dems_mode)

Dem supported issues: ['Physician Fee', 'El Salvador Aid', 'Religion-Schools', 'Education Spending', 'Superfund Right To Sue', 'Crime']
Rep supported issues: ['Disabled Infants', 'Budget Resolution', 'Anti-Satellite Test', 'Nicaragua Contras Aid', 'MX Missile', 'Synfuels Cutback', 'Duty Free Exports']
Bipartisan issues: ['Water Project', 'Immigration']
